In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scripts.forest import Forest
from sklearn.ensemble import IsolationForest
from sklearn import mixture
from scipy.linalg import qr
import scipy.stats as scp
from math import cos

In [2]:
def add_dimension(arr, dim):
    if dim == 0:
        return arr
    _, n_pts = np.shape(arr)
    added = np.random.randint(low=0, high = 100, size = ( dim, n_pts))
    added = added - 50
    added = added/25
    return np.vstack((arr,added))
    
    

In [3]:
#n_pts per Gaussian
def sample_points(n_Gaussians, n_pts, noisy_dim):
    means = []
    covs = []
    for i in range(n_Gaussians):
        #a = np.random.randint(0, 3 * n_Gaussians)
        a = 4*i
        #b = np.random.randint(0, 3 * n_Gaussians)
        b = 4*i
        c = np.random.randint(0, 1000)
        means.append([a,b])
        cov = [[1,cos(c)], [cos(c), 2]]
        #cov = [[0.3 + 3*i,0.1],[0.1,0.1 + 3*i]]
        covs.append(cov)
    G_x = np.zeros((n_Gaussians, n_pts))
    G_y = np.zeros((n_Gaussians, n_pts))
    pts_l = []
    for i in range(n_Gaussians):
        G_x[i], G_y[i] = np.random.multivariate_normal(means[i], covs[i], n_pts).T
        pts_l.append(np.vstack((G_x[i], G_y[i])))    
    pts = np.hstack(pts_l)
    noisy_pts = add_dimension(pts, noisy_dim)
    return pts, noisy_pts, means, covs
    
    

In [4]:
def get_truth(pts, means, covs):    
    dist = []
    n_G = len(means[0])
    for i in range(n_G):
        dist.append(scp.multivariate_normal(mean = means[i], cov = covs[i]))
    ptsT = np.transpose(pts)
    t_scores = np.zeros(len(ptsT))
    for i in range(len(t_scores)):
        for j in range(n_G):
            t_scores[i] += dist[j].pdf(ptsT[i])
    t_indices = np.argsort(t_scores)[:100]
    #plt.figure(figsize=(10,10))
    #plt.title("ground truth")
    #plt.plot(pts[0,:], pts[1,:], 'ro')
    #plt.plot(pts[0,t_indices], pts[1,t_indices], 'go')
    #plt.savefig("mixture_ground_truth.pdf")
    #plt.show()
    return t_indices


In [5]:
def get_iso(noisy_pts):
    rng = np.random.RandomState(27)
    _, n_p = np.shape(noisy_pts)
    clf = IsolationForest(max_samples = 100, random_state = rng, contamination = 0.1, n_estimators= int(n_p / 50), behaviour = "new")
    clf.fit(np.transpose(noisy_pts))
    Y = clf.predict(np.transpose(noisy_pts))
    iso_indices = []
    for i in range(len(Y)):
        if Y[i] == -1:
            iso_indices.append(i)
    return iso_indices

def get_lof(noisy_pts):
    lof = LocalOutlierFactor(n_neighbors=10, contamination=0.1)
    Z = lof.fit_predict(np.transpose(noisy_pts))
    lof_indices = []
    for i in range(len(Z)):
        if Z[i] == -1:
            lof_indices.append(i)
    return lof_indices

def get_density(noisy_pts):
    _, n_p = np.shape(noisy_pts)
    kwargs = {'max_depth': 20, 'n_trees':int(n_p / 50),  'max_samples': 100, 'max_buckets': 2, 'epsilon': 0.1, 'sample_axis': 1}
    forest = Forest(**kwargs)
    forest.fit(noisy_pts)
    gsw_indices, outliers, scores , pst = forest.predict(noisy_pts, 0.1)
    return gsw_indices

def get_em(noisy_pts, n_g):
    mlf = mixture.GaussianMixture(n_components=n_g, covariance_type='full')
    mlf.fit(np.transpose(noisy_pts))
    Z = -mlf.score_samples(np.transpose(noisy_pts))
    mix_indices = np.argsort(Z)[-100:]
    return mix_indices

In [6]:
def create_figure(pts, indices, fig_title):
    plt.figure(figsize=(10,10))
    plt.title(fig_title)
    plt.plot(pts[0,:], pts[1,:], 'ro')
    plt.plot(pts[0,indices], pts[1,indices], 'go')
    return plt

In [7]:
n_Gaussians = 2
noisy_dim = 2
n_pts = 500


In [8]:
n_exp = 10
dimensions = 20
final_res = np.zeros((3, dimensions))
for noisy_dim in range(dimensions):
    print("n_G: ", n_Gaussians, "noisy dim: ", noisy_dim)
    res = np.zeros((3, n_exp))
    for exp in range(n_exp):    
        pts, noisy_pts, means, covs = sample_points(n_Gaussians, n_pts, noisy_dim)
        t_indices = get_truth(pts, means, covs)
        iso_indices = get_iso(noisy_pts)
        gsw_indices = get_density(noisy_pts)
        em_indices = get_em(noisy_pts, n_Gaussians)
        us = len(set(t_indices).intersection(set(gsw_indices)))
        iso = len(set(t_indices).intersection(set(iso_indices)))
        em = len(set(em_indices).intersection(set(t_indices)))

        res[:, exp] = [us, iso, em]
        print("Exp: ", exp, "Results [us, iso , em]", us, iso, em)
    final_res[:, noisy_dim] = [np.average(res[0,:]), np.average(res[1,:]), np.average(res[2,:]) ]
    print("averages for dimension", noisy_dim, ": ", final_res[:,noisy_dim])
    

n_G:  2 noisy dim:  0


KeyError: 'threshold'

In [ ]:
#plt.figure(figsize=(20,10))
plt.title('Mixture of Gaussians with added Noise')
plt.plot(final_res[0,:], label = "density forest", linewidth = 2, marker = "v")
plt.plot(final_res[1,:], label = "isolation forest", linewidth = 2, marker = "o")
plt.plot(final_res[2,:], label = "EM mixture estimation", linewidth = 2, marker = "s")
plt.xticks(np.arange(0, 20, step=2))
plt.legend()
plt.ylabel('accuracy')
#plt.xlabel('noise dimension')
plt.axis([0, 20, 0, 100])
plt.show()

In [ ]:
with open(mixture.txt,'w') as f:
    f.write(final_res)

In [ ]:
print(final_res)